In [9]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

import matplotlib.pyplot as plt

import sys
sys.path.insert(0, "../")
from SC_Utils.game_utils import *
from pysc2.lib import actions 

# Generalizing A2C inspection

In [2]:
action_names = ['no_op', 'select_army', 'Attack_screen', 'Move_screen', 'select_point', 'select_rect']
action_dict = get_action_dict(action_names)
action_dict

{0: <_Functions.no_op: 0>,
 1: <_Functions.select_army: 7>,
 2: <_Functions.Attack_screen: 12>,
 3: <_Functions.Move_screen: 331>,
 4: <_Functions.select_point: 2>,
 5: <_Functions.select_rect: 3>}

In [7]:
# Environment parameters
RESOLUTION = 32
MAX_STEPS = 256
game_params = dict(feature_screen=RESOLUTION, feature_minimap=RESOLUTION, action_space="FEATURES") 
game_names = {1:'MoveToBeacon',
              2:'CollectMineralShards',
              3:'DefeatRoaches',
              4:'FindAndDefeatZerglings',
              5:'DefeatZerglingsAndBanelings',
              6:'CollectMineralsAndGas',
              7:'BuildMarines'
              }
map_name = game_names[2]

# Observation Processer parameters
screen_names = ['visibility_map', 'player_relative', 'selected', 'unit_density', 'unit_density_aa']
minimap_names = []
obs_proc_params = {'screen_names':screen_names, 'minimap_names':minimap_names}
#obs_proc_params = {'select_all':True}

In [10]:
test_env = init_game(game_params, map_name, max_steps=MAX_STEPS)

In [11]:
all_actions = test_env.action_spec()[0][1]
all_arguments = test_env.action_spec()[0][0]

In [21]:
arg_names = []
arg_ids = []
for a in action_dict:
    args = all_actions[action_dict[a]].args
    for arg in args:
        arg_names.append(arg.name)
        arg_ids.append(arg.id)
        
unique_args, unique_idx = np.unique(arg_names, return_index=True)
arg_ids = np.array(arg_ids)
unique_ids = arg_ids[unique_idx]
print('arg_names: ', arg_names)
print('unique_args: ', unique_args)
print("unique_ids: ", unique_ids)

spatial = []
for i, arg in enumerate(unique_args):
    size = all_arguments[unique_ids[i]].sizes
    if len(size) == 1:
        spatial.append(False)
    else:
        spatial.append(True)
print("spatial: ", spatial)

arg_names:  ['select_add', 'queued', 'screen', 'queued', 'screen', 'select_point_act', 'screen', 'select_add', 'screen', 'screen2']
unique_args:  ['queued' 'screen' 'screen2' 'select_add' 'select_point_act']
unique_ids:  [3 0 2 7 6]
spatial:  [False, True, True, False, False]


# Spatial parameters sampling

Take as input a batch of matrices of logits (B, L, L), where L is the linear size of the map.
Returns [[x_1,y_1],...,[x_B, y_B]] (or [y,x]) integer coordinates of the sampled pixels, toghether with their log probs (shape (B,)) and the probs of all the pixels (shape (B,L^2)).
Note: Softmax is performed inside

In [26]:
def old_version(x, L, x_first=True, debug=True):
    B = x.shape[0]
    ### usually self.size instead of size and it is already known
    size = L
    ###
    if debug: print("x.shape: ", x.shape)
    x = x.reshape((x.shape[0],-1))
    if debug: print("x.shape: ", x.shape)
    log_probs = F.log_softmax(x, dim=(-1))
    probs = torch.exp(log_probs)
    if debug: 
        print("log_probs.shape: ", log_probs.shape)
        print("log_probs.shape (reshaped): ", log_probs.view(-1, size, size).shape)

    # assume squared space
    x_lin = torch.arange(size).unsqueeze(0)
    xx = x_lin.repeat(B,size,1)
    if debug: print("xx.shape: ", xx.shape)
    # yx 
    args = torch.cat([xx.view(-1,size,size,1), xx.permute(0,2,1).view(-1,size,size,1)], axis=3)
    if debug: print("args.shape (before reshaping): ", args.shape)
    args = args.reshape(B,-1,2)
    if debug: print("args.shape (after reshaping): ", args.shape)
    #print("args (after reshape): ", args)
    index = Categorical(probs).sample()
    arg = args[torch.arange(B), index].detach().numpy() # and this are the sampled coordinates
    #print("index: ", index) 
    arg_lst = [list(a)  for a in arg] # swap to xy
    #print("arg_lst: ", arg_lst)
    log_probs = log_probs.reshape(B, size, size)
    # CORRECT
    return arg_lst, log_probs[torch.arange(B), arg[:,1], arg[:,0]], probs 
    # WRONG
    # return arg_lst, log_probs[torch.arange(B), arg[:,0], arg[:,1]], probs 

In [22]:
def working_version(x, L, x_first=True, debug=True):
    ### usually self.size instead of size and it is already known
    size = L
    ###
    B = x.shape[0]
    if debug: print("x.shape: ", x.shape)
    x = x.reshape((x.shape[0],-1))
    if debug: print("x.shape: ", x.shape)
    log_probs = F.log_softmax(x, dim=(-1))
    probs = torch.exp(log_probs)
    if debug: 
        print("log_probs.shape: ", log_probs.shape)
        print("log_probs.shape (reshaped): ", log_probs.view(-1, size, size).shape)

    # assume squared space
    x_lin = torch.arange(size).unsqueeze(0)
    xx = x_lin.repeat(B,size,1)
    if debug: print("xx.shape: ", xx.shape)
    # yx 
    args = torch.cat([xx.permute(0,2,1).view(-1,size,size,1), xx.view(-1,size,size,1)], axis=3)
    if debug: print("args.shape (before reshaping): ", args.shape)
    args = args.reshape(B,-1,2)
    if debug: print("args.shape (after reshaping): ", args.shape)
    #print("args (after reshape): ", args)
    index = Categorical(probs).sample()
    arg = args[torch.arange(B), index].detach().numpy() # and this are the sampled coordinates
    #print("index: ", index) 
    arg_lst = [list([a[1],a[0]])  for a in arg] # swap to xy
    #print("arg_lst: ", arg_lst)
    log_probs = log_probs.reshape(B, size, size)
    return arg_lst, log_probs[torch.arange(B), arg[:,0], arg[:,1]], probs 

In [32]:
def unravel_index(index, shape):
    out = []
    for dim in reversed(shape):
        out.append(index % dim)
        index = index // dim
    return tuple(reversed(out))

In [52]:
def clean_version(x, L, x_first=True, debug=True):
    size = L
    B = x.shape[0]
    x = x.reshape((x.shape[0],-1))
    log_probs = F.log_softmax(x, dim=(-1))
    probs = torch.exp(log_probs)
    index = Categorical(probs).sample()
    print("index: ", index)
    y, x = unravel_index(index, (size,size))
    print("y, x: ", y, x)
    arg_lst = [[xi.item(),yi.item()] for xi, yi in zip(x,y)]
    log_prob = log_probs[torch.arange(B), index]
    return arg_lst, log_prob, probs

## Test1

In [54]:
B = 1
L = 4
torch.manual_seed(1)
value = [[0.,1.,1.,0.],
         [0.,0.,0.,0.],
         [0.,0.,0.,0.],
         [0.,0.,0.,0.]]
logits = torch.tensor([value,value])
mask = (logits==0)
logits = logits.masked_fill((mask).bool(), float('-inf'))
logits = logits.reshape((logits.shape[0],-1))
print(logits)

# x first result should be (2,0)

tensor([[-inf, 1., 1., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, 1., 1., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]])


In [55]:
arg_lst, log_prob, probs = old_version(logits, L)
print("arg_lst: ", arg_lst)
print("log_prob: ", log_prob)
print("probs: ", probs)

x.shape:  torch.Size([2, 16])
x.shape:  torch.Size([2, 16])
log_probs.shape:  torch.Size([2, 16])
log_probs.shape (reshaped):  torch.Size([2, 4, 4])
xx.shape:  torch.Size([2, 4, 4])
args.shape (before reshaping):  torch.Size([2, 4, 4, 2])
args.shape (after reshaping):  torch.Size([2, 16, 2])
arg_lst:  [[1, 0], [1, 0]]
log_prob:  tensor([-0.6931, -0.6931])
probs:  tensor([[0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])


In [56]:
arg_lst, log_prob, probs = working_version(logits, L)
print("arg_lst: ", arg_lst)
print("log_prob: ", log_prob)
print("probs: ", probs)

x.shape:  torch.Size([2, 16])
x.shape:  torch.Size([2, 16])
log_probs.shape:  torch.Size([2, 16])
log_probs.shape (reshaped):  torch.Size([2, 4, 4])
xx.shape:  torch.Size([2, 4, 4])
args.shape (before reshaping):  torch.Size([2, 4, 4, 2])
args.shape (after reshaping):  torch.Size([2, 16, 2])
arg_lst:  [[1, 0], [1, 0]]
log_prob:  tensor([-0.6931, -0.6931])
probs:  tensor([[0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])


In [57]:
clean_version(logits, L)

index:  tensor([2, 1])
y, x:  tensor([0, 0]) tensor([2, 1])


([[2, 0], [1, 0]],
 tensor([-0.6931, -0.6931]),
 tensor([[0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]))